In [1]:
'''
Plotting libraries
'''
import pandas as pd
import matplotlib.cm as cm
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from time import  time
%matplotlib inline

'''
What we'll need for analysis, clustering, etc.
'''
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn import datasets, decomposition
from sklearn.manifold import TSNE

'''
Of course the powerful RDKIT for cheminformatics 
'''
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys, Descriptors, Descriptors3D, Draw, rdMolDescriptors, Draw, PandasTools
from rdkit.DataManip.Metric.rdMetricMatrixCalc import GetTanimotoSimMat, GetTanimotoDistMat
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmilesFromSmiles
from rdkit.Chem import PandasTools
'''
Some utilities
'''
import progressbar
from math import pi

%config Completer.use_jedi = False
PandasTools.RenderImagesInAllDataFrames(images=True)
IPythonConsole.ipython_useSVG=True 
import mols2grid
from tqdm import tqdm, tqdm_notebook
from ipywidgets import widgets


In [3]:
data_task = r'C:\Users\meryc\Dropbox\Toxicidade aquática\CURATED\matrix_aquatic.csv'

data_task = pd.read_csv(data_task, delimiter=',', low_memory=False)

data_task

,ID,SMILES,pLC50danio_rerio,pLC50cyprinus_carpio,pLC50epomis_macrochiru,pLC50oncorhynchus_myki,pLC50oryzias_latipe,pLC50pimephales_promela,pLC50poecilia_reticulata
0,0,C1=CC(=CC=C1[N+](=O)[O-])Cl,0.544223,NaN,NaN,NaN,NaN,NaN,NaN
1,1,C1=CC(=CC=C1N)[N+](=O)[O-],0.197770,NaN,-2.589051,-2.079964,NaN,-3.563994,NaN
2,2,C1=CC(=CC=C1[N+](=O)[O-])O,0.720114,0.702451,-2.199063,-2.362151,NaN,NaN,NaN
3,3,Cl[Se](Cl)(Cl)Cl,0.619789,NaN,NaN,NaN,NaN,NaN,NaN
4,4,ClCC1C=CC=CC=1,1.500323,NaN,NaN,NaN,NaN,-0.228518,NaN
...,...,...,...,...,...,...,...,...,...
2894,2894,C1=CSC2=CC=CC=C21,NaN,NaN,NaN,NaN,NaN,NaN,0.994199
2895,2895,OC1=CC(O)=C(Cl)C=C1,NaN,NaN,NaN,NaN,NaN,NaN,-0.982349
2896,2896,Oc1c(Br)cc(cc1Br)[N+]([O-])=O,NaN,NaN,NaN,NaN,NaN,NaN,1.413761
2897,2897,Cc1ccc(cc1C)[N+]([O-])=O,NaN,NaN,NaN,NaN,NaN,NaN,1.213853


# Build the dense matrix (to multi-task)

In [4]:
count = 0

space = {"SMILES":[],"TASKS":[],"VALUE":[]}

col = [col for col in data_task.columns]
col = col[2:]
#col[i+3]
#space = {k:[] for k in col} 

for j in range(data_task.shape[0]):
    
    i =  data_task.shape[1]
    r_col = [v for v in data_task.iloc[j,2:i]]
    
    if None not in r_col or np.nan not in r_col :
        
        for i,v in enumerate(r_col):
            
                if v == v: # nan is never equal to itself for some reason
                    
                    space["TASKS"].append(col[i])
                    space["VALUE"].append(v)
                    space["SMILES"].append(data_task["SMILES"][j])
  
dense_df = pd.DataFrame(space)

data_space = dense_df
data_space.head(100)

,SMILES,TASKS,VALUE
0,C1=CC(=CC=C1[N+](=O)[O-])Cl,pLC50danio_rerio,0.544223
1,C1=CC(=CC=C1N)[N+](=O)[O-],pLC50danio_rerio,0.197770
2,C1=CC(=CC=C1N)[N+](=O)[O-],pLC50epomis_macrochiru,-2.589051
3,C1=CC(=CC=C1N)[N+](=O)[O-],pLC50oncorhynchus_myki,-2.079964
4,C1=CC(=CC=C1N)[N+](=O)[O-],pLC50pimephales_promela,-3.563994
...,...,...,...
95,CC1=CC(O)=CC=C1,pLC50oncorhynchus_myki,-1.915405
96,CC1=CC(O)=CC=C1,pLC50pimephales_promela,-2.980546
97,NC1=CC(Cl)=CC=C1,pLC50danio_rerio,1.273250
98,NC1=CC(Cl)=CC=C1,pLC50pimephales_promela,-2.512999


In [5]:
data_space.shape # to confirm if the shape is higher 

(5523, 3)

# Reduction of dimensionality: UMAP, t-SNE and PCA

In [6]:
import os
import time
from typing import List

import seaborn as sns
import matplotlib.pyplot as plt

from rdkit import Chem, DataStructs
from rdkit.Chem.rdchem import Mol
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

 ## Calculate computational descriptor-related to SMILES data set

In [7]:
def get_largest_fragment_from_smiles(s: str):
    mol = Chem.MolFromSmiles(s)
    if mol:
        clean_mol = LargestFragmentChooser().choose(mol)
        return Chem.MolToSmiles(clean_mol)
    return None

def compute_ecfp_descriptors(smiles_list: data_space["SMILES"]):
    """ Computes ecfp descriptors """
    
    keep_idx = []
    descriptors = []
    for i, smiles in enumerate(smiles_list):
        ecfp = _compute_single_ecfp_descriptor(smiles)
        if ecfp is not None:
            keep_idx.append(i)
            descriptors.append(ecfp)

    return np.vstack(descriptors), keep_idx

def _compute_single_ecfp_descriptor(smiles: str):
    try:
        mol = Chem.MolFromSmiles(smiles)
    except Exception as E:
        return None

    if mol:
        fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        return np.array(fp)
    
    return None

In [8]:
# Compute desrciptors and keep track of which failed to featurize
smiles2 = list(data_space["SMILES"])
smi2=[Chem.MolFromSmiles(x) for x in smiles2]

#Calculate descriptors
fps  = [AllChem.GetMorganFingerprintAsBitVect(x, 3, nBits=1024) for x in smi2] # Morgan
#fps =  [MACCSkeys.GenMACCSKeys(x) for x in smi] # MACCSKeys

ecfp_descriptors, keep_idx = compute_ecfp_descriptors(data_space["SMILES"])

# Only keep those that sucessfully featurized
data_set = data_space.iloc[keep_idx]

Reduction of dimensionality: **Calculate computational cost per reduction method with descriptor-related to SMILES data set**

In [17]:
%%time
umap_model = umap.UMAP(metric = "jaccard",
                      n_neighbors = 150,
                      n_components = 2,
                      low_memory = False,
                      min_dist = 0.01)
X_umap = umap_model.fit_transform(fps)
data_space["UMAP_0"], data_space["UMAP_1"] = X_umap[:,0], X_umap[:,1]

C:\ProgramData\Anaconda3\lib\site-packages\umap\umap_.py:1802: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\pynndescent\pynndescent_.py:906: UserWarning: Failed to correctly find n_neighbors for some samples.Results may be less than ideal. Try re-running withdifferent parameters.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\pynndescent\pynndescent_.py:1532: RuntimeWarning: invalid value encountered in correct_alternative_jaccard
  self._distance_correction(self._neighbor_graph[1]),


Wall time: 24.9 s


In [10]:
%%time
pca = PCA(n_components=2)
X_pca = pca.fit_transform(ecfp_descriptors)
data_space["PCA_0"], data_space["PCA_1"] = X_pca[:,0], X_pca[:,1]

Wall time: 382 ms


In [11]:
%%time
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(ecfp_descriptors)
data_space["TSNE_0"], data_space["TSNE_1"] = X_tsne[:,0], X_tsne[:,1]

Wall time: 37.2 s


In [12]:
# Silence non-critical RDKit warnings to minimize unnecessary outputs
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

# Calculate descriptors - Smiles

In [13]:
smiles = list(data_space["SMILES"])
smi=[Chem.MolFromSmiles(x) for x in smiles]

#Calculate descriptors
fps  = [AllChem.GetMorganFingerprintAsBitVect(x, 3, nBits=1024) for x in smi] # ECFP4
#fps =  [MACCSkeys.GenMACCSKeys(x) for x in smi] # MACCSKeys
tanimoto_sim_mat_lower_triangle=GetTanimotoSimMat(fps) #This compute a similartity matrix between all the molecules
n_mol = len(fps)
similarity_matrix = np.ones([n_mol,n_mol])
i_lower= np.tril_indices(n=n_mol,m=n_mol,k=-1)
i_upper= np.triu_indices(n=n_mol,m=n_mol,k=1)
similarity_matrix[i_lower] = tanimoto_sim_mat_lower_triangle
similarity_matrix[i_upper] = similarity_matrix.T[i_upper] 
distance_matrix = np.subtract(1,similarity_matrix) #This is the similarity matrix of all vs all molecules in our table

# Reduction of dimensionality - wiht t-SNE #

In [19]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16] # To explore the "best" number of cluster to clasify our molecules
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = kmeans.fit_predict(data_space[['PCA_0','PCA_1']])
    silhouette_avg = silhouette_score(data_space[['PCA_0','PCA_1']], cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg) #This will print the silhouette score, as higher as our data is better distributed inside the clusters

For n_clusters = 2 The average silhouette_score is : 0.41622571970537703
For n_clusters = 3 The average silhouette_score is : 0.4662175458152048
For n_clusters = 4 The average silhouette_score is : 0.4483315035512236
For n_clusters = 5 The average silhouette_score is : 0.4027667128355404
For n_clusters = 6 The average silhouette_score is : 0.3910343757741678
For n_clusters = 7 The average silhouette_score is : 0.37765474009163064
For n_clusters = 8 The average silhouette_score is : 0.3873170426977886
For n_clusters = 9 The average silhouette_score is : 0.38640689335513034
For n_clusters = 10 The average silhouette_score is : 0.39170624361581996
For n_clusters = 11 The average silhouette_score is : 0.39548349184347664
For n_clusters = 12 The average silhouette_score is : 0.3837705104371647
For n_clusters = 14 The average silhouette_score is : 0.37289052506841536
For n_clusters = 15 The average silhouette_score is : 0.3689904761754597
For n_clusters = 16 The average silhouette_score is :

In [22]:
kmeans = KMeans(n_clusters=3, random_state=10) # We define the best number of clusters (3)
clusters = kmeans.fit(data_space[['PCA_0','PCA_1']]) #TC1vs TC2

data_space['Cluster'] = pd.Series(clusters.labels_, index=data_space.index)
data_space.to_excel(r"C:\Users\meryc\Downloads\UMAP_space_aquatic.xlsx") #The tSNE table now contains the numer of cluster for each element

In [ ]:
data_space

In [39]:
from IPython.display import SVG
import plotly.express as px

df = data_space

fig = px.scatter_3d(
    df, x=df['PCA_0'], y=df['PCA_1'], z=df['Cluster'], color=df['TASKS'],   
    color_continuous_scale=['green','red'], 
    labels={'0': 'PCA_0', '1': 'PCA_1', '2': 'Cluster',},
    
)

fig.update_traces(marker_size=2)
fig.show()

In [47]:
from IPython.display import SVG
import plotly.express as px
import plotly.graph_objects as go

df = data_space

fig = go.Figure(data=go.Scatter3d(
    x=df['UMAP_0'],
    y=df['UMAP_1'],
    z=df['Cluster'],
    text=df['TASKS'],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=0.1,
        size=df['Cluster'],
        color = df['Cluster'],
        color_continuous_scale=['green','red'],
        colorbar_title = 'Cluster<br>k-means',
        line_color='rgb(140, 140, 170)'
    )
))


fig.update_layout(height=800, width=800,
                  title='Share chemical tasks information')

fig.show()

ValueError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Bad property path:
color_continuous_scale
      ^^^^^^^^^^

In [ ]:
from IPython.display import SVG
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, no_update
import plotly.graph_objects as go

df = data_space
X = data_space[['UMAP_0', 'UMAP_1', 'Cluster']]

pca = PCA(n_components=3)
components = pca.fit_transform(X)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=df['Cluster'],   
    color_continuous_scale=['green','red'], 
    labels={'0': 'UMAP_0', '1': 'UMAP_1', '2': 'Cluster',}
)

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])


@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]

    df_row = df.iloc[num]
    img_src = df_row['IMG_URL']
    name = df_row['NAME']
    form = df_row['FORM']
    desc = df_row['DESC']
    if len(desc) > 300: desc = desc[:100] + '...'

    children = [
        html.Div(children=[
            html.Img(src=img_src, style={"width": "100%"}),
            html.H2(f"{name}", style={"color": "darkblue"}),
            html.P(f"{form}"),
            html.P(f"{desc}"),
        ],
        style={'width': '200px', 'white-space': 'normal'})
    ]

    return True, bbox, children


if __name__ == "__main__":
    app.run_server(debug=True, mode='inline')


fig.update_traces(marker_size=4)
fig.show()